In [51]:
import requests
import random
import json

base_url = "http://localhost:8081"

resp = requests.get(base_url)
assert resp.status_code == 404
assert resp.json()["status"] == 404
assert resp.json()["error"] == "Not Found"


In [52]:

resp = requests.get("%s/api/docs/swagger.json"%base_url)
assert resp.status_code == 200
assert resp.json()["swagger"] == "2.0"
assert resp.json()["paths"] != None

swaggerJson = resp.json()
swaggerJson["paths"].keys()

jsonUrl = [f for f in swaggerJson["paths"].keys() if ".json" in f][0]
jsonUrl

resp = requests.get("%s%s"%(base_url, jsonUrl))
assert resp.status_code == 200


In [53]:
resp.json()

{'TrainingCertificate': {'$id': '#/properties/TrainingCertificate',
  'type': 'object',
  'title': 'The TrainingCertificate Schema',
  'required': ['name', 'contact'],
  'properties': {'name': {'type': 'string'},
   'trainingTitle': {'type': 'string'},
   'contact': {'type': 'string'},
   'date': {'type': 'string', 'format': 'date'},
   'note': {'type': 'string'}}}}

In [54]:
entities = list(resp.json().keys())
print("Available entities ", entities)

Available entities  ['TrainingCertificate']


In [55]:
entity_name=entities[0]
print("Using entity %s"%entity_name)


Using entity TrainingCertificate


In [57]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/%s'%entity_name), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module"
    
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"][entity_name]["osid"]


In [58]:
resp.json()


{'id': 'sunbird-rc.registry.create',
 'ver': '1.0',
 'ets': 1646652397818,
 'params': {'resmsgid': '',
  'msgid': '0667696d-896f-4064-8686-c4b714fe5b26',
  'err': '',
  'status': 'SUCCESSFUL',
  'errmsg': ''},
 'responseCode': 'OK',
 'result': {'TrainingCertificate': {'osid': '1-7c10c85e-fcc1-4e18-8b06-c66aa5148e4d'}}}

In [59]:
print(resp.json())
resp.status_code, userId


{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1646652397818, 'params': {'resmsgid': '', 'msgid': '0667696d-896f-4064-8686-c4b714fe5b26', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'TrainingCertificate': {'osid': '1-7c10c85e-fcc1-4e18-8b06-c66aa5148e4d'}}}


(200, '70598995414')

In [60]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


{"@context": ["https://www.w3.org/2018/credentials/v1", "https://gist.githubusercontent.com/dileepbapat/eb932596a70f75016411cc871113a789/raw/498e5af1d94784f114b32c1ab827f951a8a24def/skill"], "type": ["VerifiableCredential"], "issuanceDate": "2021-08-27T10:57:57.237Z", "credentialSubject": {"type": "Person", "name": "Sunbird Learner", "trainedOn": "Sunbird RC Certificate Module"}, "issuer": "did:web:sunbirdrc.dev/vc/skill", "proof": {"type": "Ed25519Signature2018", "created": "2022-03-07T11:26:37Z", "verificationMethod": "did:india", "proofPurpose": "assertionMethod", "jws": "eyJhbGciOiJFZERTQSIsImI2NCI6ZmFsc2UsImNyaXQiOlsiYjY0Il19..Voowx39BHup_ZNmOw5flbTwM2cNX36WPkRIb7058OUZ6ipMgs5f6wah297pUnBZIbxKfTcKaIKOvQsNUVe31AQ"}}


In [44]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf"})


In [45]:
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample.pdf', 'wb') as f:
    f.write(resp.content)
    

In [46]:
%system open 'sample.pdf'


[]

In [47]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"text/html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("<html")
with open('sample.html', 'wb') as f:
    f.write(resp.content)
%system open 'sample.html'


[]

In [48]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"image/svg+xml"})
resp.status_code, resp.content

assert resp.content.decode().find("<svg") != -1
with open('sample.svg', 'wb') as f:
    f.write(resp.content)
%system open 'sample.svg'

[]

# generate certificate by external templates

In [49]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/pdf", "template": "https://raw.githubusercontent.com/dileepbapat/ref-sunbirdrc-certificate/main/schemas/templates/TrainingCertificate.html"})
resp.status_code, resp.content

assert resp.content[:5].decode().startswith("%PDF")
with open('sample_external.pdf', 'wb') as f:
    f.write(resp.content)
%system open 'sample_external.pdf'


[]

# delete certificate

In [50]:
esp = requests.delete("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/json"})
print(resp.status_code)
assert resp.status_code == 200
resp = requests.get("%s/api/v1/%s/%s"%(base_url, entity_name, idx), headers={"Accept":"application/json"})
print(json.dumps(resp.json()))
assert resp.json()["params"]["errmsg"] == "entity status is inactive"


200
{"id": "sunbird-rc.registry.read", "ver": "1.0", "ets": 1646624071473, "params": {"resmsgid": "", "msgid": "45967509-87eb-4dc4-997d-777c251dbc00", "err": "", "status": "UNSUCCESSFUL", "errmsg": "entity status is inactive"}, "responseCode": "OK"}


# generate credentials with external template

In [90]:
userId =str(random.randint(1e10,1e11))
resp = requests.post("%s%s"%(base_url, '/api/v1/SkillCertificate'), json={
    "name":"Sunbird Learner", 
    "contact": userId, 
    "trainingTitle":"Sunbird RC Certificate Module",
    "skills": [
        {
            "skill": "computer training",
            "certifiedOn": "12-12-2000"
        },
        {
            "skill": "network training",
            "certifiedOn": "12-11-2000"
        }

    ]
    
})
assert resp.status_code == 200 or print (resp.json())
idx = resp.json()["result"]["SkillCertificate"]["osid"]
print(resp.json())

{'id': 'sunbird-rc.registry.create', 'ver': '1.0', 'ets': 1646657562144, 'params': {'resmsgid': '', 'msgid': '78cc1584-93dc-41ec-bac7-3dd960f812df', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'SkillCertificate': {'osid': '1-695be36f-939b-4e0d-bf89-969eff6f2261'}}}


In [91]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, 'SkillCertificate', idx), headers={"Accept":"application/vc+ld+json"})
print(json.dumps(resp.json()))
assert resp.json()["proof"]["type"] == "Ed25519Signature2018"


{"@context": ["https://www.w3.org/2018/credentials/v1", {"@context": {"@version": 1.1, "@protected": true, "SkillCertificate": {"@id": "https://github.com/sunbird-specs/vc-specs#SkillCertificate", "@context": {"id": "@id", "@version": 1.1, "@protected": true, "skills": "schema:Text"}}, "Person": {"@id": "https://github.com/sunbird-specs/vc-specs#Person", "@context": {"name": "schema:Text"}}, "trainedOn": {"@id": "https://github.com/sunbird-specs/vc-specs#trainedOn", "@context": {"name": "schema:Text"}}, "skills": {"@id": "https://github.com/sunbird-specs/vc-specs#skills", "@container": "@list"}}}], "type": ["VerifiableCredential"], "issuanceDate": "2021-08-27T10:57:57.237Z", "credentialSubject": {"type": "Person", "name": "Sunbird Learner", "trainedOn": "Sunbird RC Certificate Module", "skills": ["computer training", "network training", ""]}, "issuer": "did:web:sunbirdrc.dev/vc/skill", "proof": {"type": "Ed25519Signature2018", "created": "2022-03-07T12:52:42Z", "verificationMethod": "d

In [108]:
resp = requests.get("%s/api/v1/%s/%s"%(base_url, 'SkillCertificate', idx), headers={"Accept":"application/pdf", "template": "http://localhost:63342/sunbird-rc/registry/public/_schemas/templates/SkillCertificate.html?_ijt=qu4536mc0j899963igug2gitf0"})

resp.status_code, resp.content
print(resp)
assert resp.content[:5].decode().startswith("%PDF")
with open('sample1.pdf', 'wb') as f:
    f.write(resp.content)
    
%system open 'sample1.pdf'


<Response [200]>


[]